In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os.path

In [3]:
# out_path = "/work/scratch/malinoro/simulation_copula/out/"
out_path = "/Users/roman/Code/simulation_copula/out"

## Multiple figures

In [4]:
def plot_possibilities(a, b):

    fig, axes = plt.subplots(1, 2, figsize=(15,5))

    axes[0].scatter(range(len(a)), a.values(), marker='+', s=10*mpl.rcParams['lines.markersize'] ** 2, c="blue")
    axes[0].set_title("X poss")
    
    axes[0].set_xticks(ticks=range(len(a)))
    axes[0].set_xticklabels(labels=list(a.keys()))
    axes[0].set_yticks(ticks=list(a.values()))
    axes[0].set_yticklabels(labels=list(a.values()))
    
    
    axes[1].scatter(range(len(b)), b.values(), marker='+', s=10*mpl.rcParams['lines.markersize'] ** 2, c="orange")
    axes[1].set_title("Y poss")
    
    axes[1].set_xticks(ticks=range(len(b)))
    axes[1].set_xticklabels(labels=list(b.keys()))
    axes[1].set_yticks(ticks=list(b.values()))
    axes[1].set_yticklabels(labels=list(b.values()))
    
    if order_a is not None or order_b is not None:
        for k in order_a.keys():
            for i in [int(i) for i in "".join(k.split(",")).split("x") if i!=""]:
                axes[2].plot([i-0.5,i+0.5], order_a[k], c="blue")
        axes[2].set_title("X Order")
        
        for k in order_b.keys():
            for i in [int(i) for i in "".join(k.split(",")).split("x") if i!=""]:
                axes[3].plot([i-0.5,i+0.5], order_b[k], c="orange")
        axes[3].set_title("Y Order")
    
    fig.show()
    
    
def plot_possibilities(a, b, order_a, order_b):
    
    fig, axes = plt.subplots(2, 2, figsize=(15,10))

    axes[0,0].scatter(range(len(a)), a.values(), marker='+', s=10*mpl.rcParams['lines.markersize'] ** 2, c="blue")
    axes[0,0].set_title("X poss")
    
    axes[0,0].set_xticks(ticks=range(len(a)))
    axes[0,0].set_xticklabels(labels=list(a.keys()))
    axes[0,0].set_yticks(ticks=list(a.values()))
    axes[0,0].set_yticklabels(labels=list(a.values()))
    
    
    axes[0,1].scatter(range(len(b)), b.values(), marker='+', s=10*mpl.rcParams['lines.markersize'] ** 2, c="orange")
    axes[0,1].set_title("Y poss")
    
    axes[0,1].set_xticks(ticks=range(len(b)))
    axes[0,1].set_xticklabels(labels=list(b.keys()))
    axes[0,1].set_yticks(ticks=list(b.values()))
    axes[0,1].set_yticklabels(labels=list(b.values()))
    
    for k in order_a.keys():
        for i in [int(i) for i in "".join(k.split(",")).split("x") if i!=""]:
            axes[1,0].plot([i-0.5,i+0.5], [order_a[k]]*2, c="blue")
    axes[1,0].set_title("X Order")
    axes[1,0].set_xticks(ticks=range(1, len(a)+1))
    axes[1,0].set_xticklabels(labels=list(a.keys()))
    axes[1,0].set_yticks(ticks=list(order_a.values()))
    axes[1,0].set_yticklabels(labels=list(order_a.values()))

    for k in order_b.keys():
        for i in [int(i) for i in "".join(k.split(",")).split("y") if i!=""]:
            axes[1,1].plot([i-0.5,i+0.5], [order_b[k]]*2, c="orange")
    axes[1,1].set_title("Y Order")
    axes[1,1].set_xticks(ticks=range(1, len(b)+1))
    axes[1,1].set_xticklabels(labels=list(b.keys()))
    axes[1,1].set_yticks(ticks=list(order_b.values()))
    axes[1,1].set_yticklabels(labels=list(order_b.values()))
    
    fig.show()
    
def generator_lines(df):
    atoms = [k[1] for k in df.columns if k[0]=="poss"]
    keys_x = [k for k in atoms if "x" in k]
    keys_y = [k for k in atoms if "y" in k]
    
    for k in df.index:
        poss_x = {i: df.loc[k, ("poss", i)] for i in keys_x}
        poss_y = {i: df.loc[k, ("poss", i)] for i in keys_y}
        
        order_x = df.loc[k, ("focal_sets", "X")].split("<")
        order_x = {order_x[i]: i + 1 for i in range(len(order_x))}
        
        order_y = df.loc[k, ("focal_sets", "Y")].split("<") 
        order_y = {order_y[i]: i + 1 for i in range(len(order_y))}
        yield [poss_x, poss_y, order_x, order_y] 

In [6]:
df_1 = pd.read_csv(os.path.join(out_path, "random_orders_N3_min.csv"), header=[0,1], index_col=[0,1])
df_2 = pd.read_csv(os.path.join(out_path, "random_orders_N3.csv"), header=[0,1], index_col=[0,1])

In [ ]:
gen = generator_lines(df)

for _ in range(len(df)):
    g = next(gen)
    print(g)
    plot_possibilities(g[0], g[1], g[2], g[3])
    break
    

# Analyse if df have the same possibilities 

In [35]:
poss = df_1.columns.get_loc_level(key="poss")[1]  # Index(['x1', 'x2', 'x3', 'y1', 'y2', 'y3'], dtype='object', name='Space')
unique_poss_1 = df_1.xs(key=0, level="order", axis=0, drop_level=False)

In [43]:
unique_poss_1.xs(key="x2", level="Space", axis=1, drop_level=False)

,Object,poss
,Space,x2
poss,order,
0,0,0.95
1,0,0.95
2,0,0.95
3,0,0.95
4,0,0.95
...,...,...
3907,0,0.94
3908,0,0.94


In [77]:
k = 2910
n = len(unique_poss_1)
unique_poss_1.loc[(n-k-10,):(n-k-1,), ("poss",)]

/var/folders/kn/x94nsr4955n7vthghm5m3mfw0000gn/T/ipykernel_52901/2636090159.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  unique_poss_1.loc[(n-k-10,):(n-k-1,), ("poss",)]


,Space,x1,x2,x3,y1,y2,y3
poss,order,,,,,,
992,0,0.14,0.95,1.0,0.49,0.85,1.00
993,0,0.14,0.95,1.0,1.00,0.75,0.23
994,0,0.14,0.95,1.0,1.00,0.73,0.42
995,0,0.42,0.83,1.0,1.00,0.00,0.25
996,0,0.42,0.83,1.0,1.00,0.00,0.50
997,0,0.42,0.83,1.0,1.00,0.00,0.75
998,0,0.42,0.83,1.0,1.00,0.25,0.00
999,0,0.42,0.83,1.0,1.00,0.25,0.25
1000,0,0.42,0.83,1.0,1.00,0.25,0.50


In [41]:
tuple([1,2,3])

(1, 2, 3)

In [50]:
unique_poss_1

Object      poss                                      focal_sets  \
Space         x1    x2   x3    y1    y2    y3                  X   
poss order                                                         
0    0      0.14  0.95  1.0  1.00  0.81  0.09  x1,x2,x3<x2,x3<x3   
1    0      0.14  0.95  1.0  1.00  0.73  0.19  x1,x2,x3<x2,x3<x3   
2    0      0.14  0.95  1.0  0.66  1.00  0.56  x1,x2,x3<x2,x3<x3   
3    0      0.14  0.95  1.0  1.00  0.43  0.67  x1,x2,x3<x2,x3<x3   
4    0      0.14  0.95  1.0  1.00  0.97  0.68  x1,x2,x3<x2,x3<x3   
...          ...   ...  ...   ...   ...   ...                ...   
3907 0      0.40  0.94  1.0  1.00  0.75  0.25  x1,x2,x3<x2,x3<x3   
3908 0      0.40  0.94  1.0  1.00  0.75  0.50  x1,x2,x3<x2,x3<x3   
3909 0      0.40  0.94  1.0  1.00  0.75  0.75  x1,x2,x3<x2,x3<x3   
3910 0      0.40  0.94  1.0  1.00  0.75  1.00  x1,x2,x3<x2,x3<x3   
3911 0      0.40  0.94  1.0  1.00  1.00  0.25  x1,x2,x3<x2,x3<x3   

Object                         
Space                       Y  
poss order                     
0    0      y1<y1,y2,y3<y1,y2  
1    0      y1<y1,y2<y1,y2,y3  
2    0      y1,y2<y2<y1,y2,y3  
3    0      y1<y1,y3<y1,y2,y3  
4    0      y1<y1,y2<y1,y2,y3  
...                       ...  
3907 0      y1<y1,y2<y1,y2,y3  
3908 0      y1<y1,y2<y1,y2,y3  
3909 0            y1<y1,y2,y3  
3910 0         y1,y3<y1,y2,y3  
3911 0         y1,y2<y1,y2,y3  

[3912 rows x 8 columns]

In [55]:
[2:10]

SyntaxError: invalid syntax (2931633738.py, line 1)